In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [12]:
import llama_index
print(llama_index.__version__)


AttributeError: module 'llama_index' has no attribute '__version__'

In [10]:
from llama_index import SimpleDirectoryReader  # Import only the available class


ImportError: cannot import name 'SimpleDirectoryReader' from 'llama_index' (unknown location)